In [19]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json
from datetime import datetime 
from alpaca_trade_api.rest import REST, TimeFrame
import time
from time import sleep
%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()
opensea_api_key = os.getenv("opensea_api_key")

In [3]:
opensea_collections_url = "https://api.opensea.io/api/v1/collections"
opensea_collection_stats_url = "https://api.opensea.io/api/v1/collection/doodles-official/stats"

In [6]:
def get_opensea_api_data(url):
    response_data = requests.get(url).json()
    print(json.dumps(response_data, indent=2))
    return response_data

In [8]:
#get_opensea_api_data(opensea_collections_url)
get_opensea_api_data(opensea_collection_stats_url)

{
  "stats": {
    "one_day_volume": 158.2678205811,
    "one_day_change": 0.13673648338073685,
    "one_day_sales": 11.0,
    "one_day_average_price": 14.38798368919091,
    "seven_day_volume": 1709.4142400295816,
    "seven_day_change": -0.6349826634695173,
    "seven_day_sales": 101.0,
    "seven_day_average_price": 16.92489346563942,
    "thirty_day_volume": 14452.315883643685,
    "thirty_day_change": 0.491459644932021,
    "thirty_day_sales": 964.0,
    "thirty_day_average_price": 14.992028924941582,
    "total_volume": 102692.81166761978,
    "total_sales": 20875.0,
    "total_supply": 10000.0,
    "count": 10000.0,
    "num_owners": 4646,
    "average_price": 4.91941612779017,
    "num_reports": 66,
    "market_cap": 169248.9346563942,
    "floor_price": 13.9
  }
}


{'stats': {'one_day_volume': 158.2678205811,
  'one_day_change': 0.13673648338073685,
  'one_day_sales': 11.0,
  'one_day_average_price': 14.38798368919091,
  'seven_day_volume': 1709.4142400295816,
  'seven_day_change': -0.6349826634695173,
  'seven_day_sales': 101.0,
  'seven_day_average_price': 16.92489346563942,
  'thirty_day_volume': 14452.315883643685,
  'thirty_day_change': 0.491459644932021,
  'thirty_day_sales': 964.0,
  'thirty_day_average_price': 14.992028924941582,
  'total_volume': 102692.81166761978,
  'total_sales': 20875.0,
  'total_supply': 10000.0,
  'count': 10000.0,
  'num_owners': 4646,
  'average_price': 4.91941612779017,
  'num_reports': 66,
  'market_cap': 169248.9346563942,
  'floor_price': 13.9}}

In [15]:
def get_events(start_date, end_date, cursor='', event_type='successful', **kwargs):
    url = "https://api.opensea.io/api/v1/events"
    query = { 
        "only_opensea": "false", 
        "occurred_before": end_date,
        "occurred_after": start_date,
        "event_type": event_type,
        "cursor": cursor,
         **kwargs
    }

    headers = {
        "Accept": "application/json",
        "X-API-KEY": opensea_api_key
    }
    response = requests.request("GET", url, headers=headers, params=query)

    return response.json()

In [10]:
def parse_event(event):
    record = {}
    asset = event.get('asset')
    if asset == None:
        return None # if there's no asset that means it's not a single NFT transaction so skip this item

    #collection
    record['collection_slug'] = asset['collection']['slug']
    record['collection_name'] = asset['collection']['name']
    record['collection_url'] = "https://opensea.io/collection/" + asset['collection']['slug']

    #asset
    record['asset_id'] = asset['id']
    record['asset_name'] = asset['name']
    record['asset_description'] = asset['description']
    record['asset_contract_date'] = asset['asset_contract']['created_date']
    record['asset_url'] = asset['permalink']
    record['asset_img_url'] = asset['image_url']

    #event
    record['event_id'] = event['id']
    record['event_time'] = event.get('created_date')
    record['event_auction_type'] = event.get('auction_type')
    record['event_contract_address'] = event.get('contract_address')
    record['event_quantity'] = event.get('quantity')
    record['event_payment_symbol'] =  None if event.get('payment_token') == None else event.get('payment_token').get('symbol')

    decimals = 18
    if event.get('payment_token') != None:
        decimals = event.get('payment_token').get('decimals')

    price_str = event['total_price']

    try: 
        if len(price_str) < decimals:
            price_str =  "0." + (decimals-len(price_str)) * "0" + price_str
            record['event_total_price'] = float(price_str)
        else:
            record['event_total_price'] = float(price_str[:-decimals] + "." + price_str[len(price_str)-decimals:])
    except:
        print(event)

    return record

In [11]:
def fetch_all_events(start_date, end_date, pause=1, **kwargs):
    result = list()
    next = ''
    fetch = True

    print(f"Fetching events between {start_date} and {end_date}")
    while fetch:
        response = get_events(int(start_date.timestamp()), int(end_date.timestamp()), cursor=next, **kwargs)

        for event in response['asset_events']:
            cleaned_event = parse_event(event)
            
            if cleaned_event != None:
                result.append(cleaned_event)

        if response['next'] is None:
            fetch = False
        else:
            next = response['next']

        sleep(pause)

    return result

In [23]:
import datetime

start_date = datetime.datetime(2022, 2, 10)
end_date = datetime.datetime(2022, 4, 10)

result = fetch_all_events(x,x)
result

Fetching events between 2020-05-17 00:00:00 and 2020-05-17 00:00:00


[]